In [3]:
import pandas as pd
import json
import requests
import math
import numpy as np

with open('config.json') as f:
    settings = json.load(f)
df = pd.read_csv("block.csv")

# Getting the distance and time Matrix
Should only be run once and the output is saved to reduce call to Google api.

In [46]:
release_pts = pd.read_excel('Yishun Wolbachia Release Points.xlsx')

In [47]:
release_pts['NumRelease'] = release_pts['ReleaseLocation'].str.split(',').apply(len)
release_pts['Block'] = release_pts['Block'].astype('str')

In [48]:
release_pts['Block'] = release_pts['Block'].str.replace('.Bin Centre', '')

In [49]:
release_pts

,Location,Postal Code,Block,ReleaseLocation,NumRelease
0,YISHUN STREET 21,760201.0,201,"51,53",2
1,YISHUN STREET 21,760201.0,201,"55,57",2
2,YISHUN STREET 21,760201.0,201,"59,61",2
3,YISHUN STREET 21,760201.0,201,"63,65",2
4,YISHUN STREET 21,760201.0,201,"67,69",2
5,YISHUN STREET 21,760202.0,202,"73,75",2
6,YISHUN STREET 21,760202.0,202,"77,79",2
7,YISHUN STREET 21,760202.0,202,"81,83",2
8,YISHUN STREET 21,760202.0,202,"85,87",2
9,YISHUN STREET 21,760202.0,202,"89,91",2


In [63]:
def count_release(df):
    output = {
        'Road' : df.iloc[0,0],
        'NumRelease' : df['NumRelease'].sum(),
    }
    return pd.Series(output)

In [64]:
df = release_pts.groupby('Block').apply(count_release).reset_index()

In [66]:
df['address'] = df['Block'] + '+' + df['Road'].str.replace(' ','+')

In [75]:
df['location'] = None

In [76]:
url= "https://maps.googleapis.com/maps/api/geocode/json"

for each in range(len(df)):
    if not df.loc[each, 'location']:
        params = dict(
            address = df.loc[each, 'address'],
            key = settings['api_key']
        )

        resp = requests.get(url=url, params=params)
        data = resp.json()

        try:
            df.loc[each, 'location'] = str(data['results'][0]['geometry']['location']['lat']) + "," + str(data['results'][0]['geometry']['location']['lng'])
        except:
            print(each)
            print(data)

In [79]:
df.to_pickle('block_with_geoloc.pickle')

In [80]:
arr,arr1,arr2,arr3,arr4 = "","", "", "", ""

arr_total = []

for index, row in df.iterrows():
    arr += str(row['address']) + "|"

for index, row in df.iterrows():
    if(index < 25):
        arr1 += str(row['address']) + "|"
        
for index, row in df.iterrows():
    if(25 <= index and index < 50):
        arr2 += str(row['address']) + "|"
        
for index, row in df.iterrows():
    if(50 <= index and index < 75):
        arr3 += str(row['address']) + "|"
    
    
for index, row in df.iterrows():
    if(75 <= index and index < 84):
        arr4 += str(row['address']) + "|"
        
arr_total.append(arr1[:-1])
arr_total.append(arr2[:-1])
arr_total.append(arr3[:-1])
arr_total.append(arr4[:-1])

arr_total

['201+YISHUN+STREET+21|202+YISHUN+STREET+21|203+YISHUN+STREET+21|204+YISHUN+STREET+21|205+YISHUN+STREET+21|206+YISHUN+STREET+21|207+YISHUN+STREET+21|208+YISHUN+STREET+21|209+YISHUN+STREET+21|210+YISHUN+STREET+21|211+YISHUN+STREET+21|212+YISHUN+STREET+21|213+YISHUN+STREET+21|214+YISHUN+STREET+21|215+YISHUN+STREET+21|216+YISHUN+STREET+21|217+YISHUN+STREET+21|218+YISHUN+STREET+21|219+YISHUN+STREET+21|220+YISHUN+STREET+21|221+YISHUN+STREET+21|222+YISHUN+STREET+21|223+YISHUN+STREET+21|224+YISHUN+STREET+21|225+YISHUN+STREET+21',
 '226+YISHUN+STREET+21|227+YISHUN+STREET+21|228+YISHUN+STREET+21|229+YISHUN+STREET+21|230+YISHUN+STREET+21|231+YISHUN+STREET+21|232+YISHUN+STREET+21|233+YISHUN+STREET+21|234+YISHUN+STREET+21|235+YISHUN+STREET+21|236+YISHUN+RING+ROAD|237+YISHUN+RING+ROAD|238+YISHUN+RING+ROAD|239+YISHUN+RING+ROAD|240+YISHUN+RING+ROAD|241+YISHUN+RING+ROAD|242+YISHUN+RING+ROAD|243+YISHUN+RING+ROAD|244+YISHUN+RING+ROAD|245+YISHUN+AVENUE+9|246+YISHUN+AVENUE+9|247+YISHUN+AVENUE+9|248+YISHUN

In [82]:
# WILL RUN FOR 2 MINS AT MAX

time_matrix = np.zeros((84,84))
time_array = []
dist_matrix = np.zeros((84,84))
dist_array = []

url_google_route = "https://maps.googleapis.com/maps/api/distancematrix/json"


for each_index in range(len(df)):
    time_array = []
    dist_array = []
    print("Inititaing " + str(each_index) + "/83 first...")
    for index, dest_input in enumerate(arr_total):
#         print("Working on " + str(index) + "/3 first...")
#         print(dest_input)
        params = dict(
            destinations = dest_input,
            origins = df.iloc[each_index]['address'],
            key = settings['api_key'],
            mode = "walking"
        )
        resp = requests.get(url=url_google_route, params=params)
        data = resp.json()
        for each in range(len(data['rows'][0]['elements'])):
            if data['rows'][0]['elements'][each]['status'] == 'OK':
                time_array.append(data['rows'][0]['elements'][each]['duration']['value'])
                dist_array.append(data['rows'][0]['elements'][each]['distance']['value'])
            else:
                print(each)
                print("Origin: {}, Destination: ".format(data['origin_addresses'][0]))
                print(data)
    print(each_index)
    time_matrix[each_index] = time_array
    dist_matrix[each_index] = dist_array


Inititaing 0/83 first...
0
Inititaing 1/83 first...
1
Inititaing 2/83 first...
2
Inititaing 3/83 first...
3
Inititaing 4/83 first...
4
Inititaing 5/83 first...
5
Inititaing 6/83 first...
6
Inititaing 7/83 first...
7
Inititaing 8/83 first...
8
Inititaing 9/83 first...
9
Inititaing 10/83 first...
10
Inititaing 11/83 first...
11
Inititaing 12/83 first...
12
Inititaing 13/83 first...
13
Inititaing 14/83 first...
14
Inititaing 15/83 first...
15
Inititaing 16/83 first...
16
Inititaing 17/83 first...
17
Inititaing 18/83 first...
18
Inititaing 19/83 first...
19
Inititaing 20/83 first...
20
Inititaing 21/83 first...
21
Inititaing 22/83 first...
22
Inititaing 23/83 first...
23
Inititaing 24/83 first...
24
Inititaing 25/83 first...
25
Inititaing 26/83 first...
26
Inititaing 27/83 first...
27
Inititaing 28/83 first...
28
Inititaing 29/83 first...
29
Inititaing 30/83 first...
30
Inititaing 31/83 first...
31
Inititaing 32/83 first...
32
Inititaing 33/83 first...
33
Inititaing 34/83 first...
34
Initi

In [84]:
np.save('time_matrix.npy', time_matrix)
np.save('dist_matrix.npy', dist_matrix)

# Loading the processed data

In [1]:
import pandas as pd
import json
import requests
import math
import numpy as np

with open('config.json') as f:
    settings = json.load(f)
df = pd.read_csv("block.csv")

In [2]:
time_matrix = np.load('time_matrix.npy')
dist_matrix = np.load('dist_matrix.npy')
df = pd.read_pickle('block_with_geoloc.pickle')

In [3]:
time_matrix[37, :] = 0

In [20]:
time_matrix[:, 37]

array([562., 602., 754., 623., 713., 732., 497., 587., 455., 497., 397.,
       454., 490., 694., 659., 502., 541., 572., 562., 621., 512., 582.,
       592., 516., 427., 468., 411., 293., 573., 454., 507., 503., 226.,
       584., 661.,  64.,  26.,   0., 121., 149., 186.,  96., 221., 277.,
       408., 487., 460., 476., 420., 426., 382., 353., 319., 273., 273.,
       143., 137., 426., 337., 335., 286., 338., 463., 260., 653., 660.,
       629., 613., 532., 541., 537., 598., 630., 643., 627., 618., 713.,
       655., 592., 307., 329., 553., 505., 368.])

In [4]:
time_matrix[:, 37] = 0 # Setting return to base time to zero.

In [8]:
time_matrix.shape

(84, 84)

In [5]:
time_matrix[35:,35:]

array([[  0.,  63.,   0., ..., 494., 437., 278.],
       [ 61.,   0.,   0., ..., 561., 503., 344.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [481., 554.,   0., ...,   0., 132., 185.],
       [433., 506.,   0., ..., 141.,   0., 137.],
       [296., 368.,   0., ..., 216., 159.,   0.]])

In [11]:
# Adding in the additional time for release
for idx, value in df['NumRelease'].iteritems():
    time_matrix[idx, :] += value*60

In [6]:
"""Vehicles Routing Problem (VRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

# Plotting the routes 

In [7]:
"""Solve the CVRP problem."""
# Instantiate the data problem.
"""Stores the data for the problem."""
data = {}
data['distance_matrix'] = time_matrix
data['num_vehicles'] = 6
data['depot'] = 37

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']), data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    100000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)

Route for vehicle 0:
 37 ->  3 ->  40 ->  39 ->  41 ->  38 ->  42 ->  79 ->  80 ->  83 ->  82 ->  81 ->  78 -> 37
Distance of the route: 1326m

Route for vehicle 1:
 37 ->  27 ->  32 ->  55 ->  43 ->  54 ->  53 ->  52 ->  49 ->  47 ->  44 ->  46 ->  45 ->  48 ->  50 ->  51 -> 37
Distance of the route: 1259m

Route for vehicle 2:
 37 ->  22 ->  21 ->  20 ->  18 ->  17 ->  16 ->  30 ->  31 ->  33 ->  34 ->  28 ->  29 ->  26 ->  63 ->  56 ->  35 ->  36 -> 37
Distance of the route: 1302m

Route for vehicle 3:
 37 ->  72 ->  71 ->  70 ->  24 ->  25 ->  23 ->  19 ->  14 ->  2 ->  4 ->  5 ->  13 ->  15 ->  12 ->  11 -> 37
Distance of the route: 1288m

Route for vehicle 4:
 37 ->  7 ->  1 ->  0 ->  9 ->  8 ->  6 ->  10 ->  73 ->  74 ->  75 ->  76 ->  77 -> 37
Distance of the route: 1165m

Route for vehicle 5:
 37 ->  69 ->  66 ->  67 ->  68 ->  62 ->  61 ->  59 ->  60 ->  58 ->  57 ->  64 ->  65 -> 37
Distance of the route: 1353m

Maximum of the route distances: 1353m


In [23]:
import folium
color = ["red", "blue", "green", "yellow", "black", "grey"]
mapa = folium.Map([1.3,103.9],
                  zoom_start=4,
                  tiles='cartodbpositron')
for vehicle_id in range(data['num_vehicles']):
    loc = []
    index = routing.Start(vehicle_id)
    print('\nRoute for vehicle {}:'.format(vehicle_id))
    route_distance = 0
    tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
    loc.append((float(tmp[0]), float(tmp[1])))
    while not routing.IsEnd(index):
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
        loc.append((float(tmp[0]), float(tmp[1])))
    folium.PolyLine(loc, color=color[vehicle_id], weight=2.5, opacity=1).add_to(mapa)
mapa


Route for vehicle 0:

Route for vehicle 1:

Route for vehicle 2:

Route for vehicle 3:

Route for vehicle 4:

Route for vehicle 5:


# Arbitrary start and end point 

In [9]:
new_time_matrix = np.zeros((85, 85))
new_time_matrix[:84,:84] = time_matrix

In [10]:
"""Solve the CVRP problem."""
# Instantiate the data problem.
"""Stores the data for the problem."""
data = {}
data['distance_matrix'] = new_time_matrix
data['num_vehicles'] = 6
data['depot'] = 84

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']), data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    100000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)

Route for vehicle 0:
 84 ->  22 ->  21 ->  33 ->  34 ->  28 ->  29 ->  26 ->  23 ->  3 -> 84
Distance of the route: 1192m

Route for vehicle 1:
 84 ->  7 ->  1 ->  0 ->  9 ->  8 ->  6 ->  10 ->  11 ->  12 ->  15 ->  16 ->  17 ->  18 ->  20 ->  31 ->  30 ->  19 ->  14 ->  13 ->  4 ->  5 ->  2 -> 84
Distance of the route: 1195m

Route for vehicle 2:
 84 ->  57 ->  58 ->  63 ->  56 ->  38 ->  55 ->  54 ->  53 ->  51 ->  50 ->  48 ->  49 ->  47 ->  45 ->  46 ->  44 -> 84
Distance of the route: 1170m

Route for vehicle 3:
 84 ->  73 ->  74 ->  75 ->  76 ->  77 ->  78 ->  81 ->  82 ->  83 ->  80 ->  79 ->  42 -> 84
Distance of the route: 1035m

Route for vehicle 4:
 84 ->  25 ->  24 ->  66 ->  67 ->  68 ->  62 ->  61 ->  60 ->  59 ->  64 ->  65 -> 84
Distance of the route: 1163m

Route for vehicle 5:
 84 ->  52 ->  43 ->  37 ->  72 ->  71 ->  70 ->  69 ->  27 ->  32 ->  35 ->  36 ->  41 ->  39 ->  40 -> 84
Distance of the route: 1175m

Maximum of the route distances: 1195m


In [13]:
import folium
color = ["red", "blue", "green", "yellow", "black", "grey"]
mapa = folium.Map([1.3,103.9],
                  zoom_start=4,
                  tiles='cartodbpositron')
for vehicle_id in range(data['num_vehicles']):
    loc = []
    index = routing.Start(vehicle_id)
    print('\nRoute for vehicle {}:'.format(vehicle_id))
    route_distance = 0
#     tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
#     loc.append((float(tmp[0]), float(tmp[1])))
    while not routing.IsEnd(index):
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        if manager.IndexToNode(index) < len(df):
            tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
            loc.append((float(tmp[0]), float(tmp[1])))
        else:
            print(manager.IndexToNode(index))
    folium.PolyLine(loc, color=color[vehicle_id], weight=2.5, opacity=1).add_to(mapa)
mapa


Route for vehicle 0:
84

Route for vehicle 1:
84

Route for vehicle 2:
84

Route for vehicle 3:
84

Route for vehicle 4:
84

Route for vehicle 5:
84
